In [ ]:
import requests
import re
from bs4 import BeautifulSoup
import csv

In [ ]:
filename = "kovo-male.csv"
f = open(filename, 'w', encoding='utf8')
writer = csv.writer(f)

In [ ]:
title = "Share_Variance Attack_Variance Open% Open_SR Delay% Delay_SR Move% Move_SR Back% Back_SR Quick% Quick_SR Quickopen% Quickopen_SR Serve_SR Dig_SR Set_SR Receive_Accuracy Receive_Variance Blocking_SR Blocking_ER Blocking_AR Penalty Error Outcome".split(' ')
writer.writerow(title)

In [ ]:
team_list = ['현대캐피탈', '삼성화재', 'KB손해보험', '한국전력', '대한항공', '상무', '우리카드', 'OK금융그룹', 'LG화재', 'LIG', 'LIG손해보험', 'KEPCO45', 'KEPCO', '신협상무', '우리캐피탈', '상무신협', '드림식스', '러시앤캐시', 'OK저축은행']

In [ ]:
def mean(alist):
    return sum(alist)/len(alist)

In [ ]:
def deviation(alist):
    average = mean(alist)
    ans = 0
    for i in alist:
        ans += (i-average)**2
    return ans/len(alist)

In [ ]:
counter = 0

for season in range(1,19):
    if season<10:
        season = "0"+str(season)
    for num in range(1,400):
        url = "https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=0"+str(season)+"&g_part=201&r_round=1&g_num="+str(num)+"&"
        print(url)
        res = requests.get(url)
        soup = BeautifulSoup(res.text,"lxml")

        team_names = soup.find_all("span", {'class':'team'})
        if team_names == []:
            break
        scores = soup.find_all('p',{'class':'num'})
        if scores[0].get_text() == scores[1].get_text():
            break
        if team_names[0].get_text() not in team_list:
            continue
        if team_names[1].get_text() not in team_list:
            continue
        
        
        team_records = soup.find_all("div", {"class":"wrp_lst"})
        for no in range(2):
            team_record = team_records[no]

            data = []
            tables = team_record.find_all('table')[1:]

            #1
            tried = []
            trs = tables[0].find('tbody').find_all('tr')
            for tr in trs:
                tds = tr.find_all('td')
                if int(tds[6].get_text()) > 0:
                    tried.append(int(tds[6].get_text()))
            data.append(deviation(tried)) #점유율 분산

            #2
            open_tried_success = [0,0]
            delay_tried_success = [0,0]
            move_tried_success = [0,0]
            trs = tables[1].find('tbody').find_all('tr')
            for tr in trs:
                tds = tr.find_all('td')
                open_tried_success[0]+=int(tds[0].get_text())
                open_tried_success[1]+=int(tds[1].get_text())
                delay_tried_success[0]+=int(tds[6].get_text())
                delay_tried_success[1]+=int(tds[7].get_text())
                move_tried_success[0]+=int(tds[12].get_text())
                move_tried_success[1]+=int(tds[13].get_text())

            data.append(open_tried_success[0]/sum(tried)*100)
            data.append(open_tried_success[1]/open_tried_success[0]*100)
            data.append(delay_tried_success[0]/sum(tried)*100)
            if delay_tried_success[0]>0:
                data.append(delay_tried_success[1]/delay_tried_success[0]*100)
            else:
                data.append('NA')
            data.append(move_tried_success[0]/sum(tried)*100)
            if move_tried_success[0]>0:
                data.append(move_tried_success[1]/move_tried_success[0]*100)
            else:
                data.append('NA')

            #3
            back_tried_success = [0,0]
            quick_tried_success = [0,0]
            quickopen_tried_success = [0,0]
            trs = tables[2].find('tbody').find_all('tr')
            for tr in trs:
                tds = tr.find_all('td')
                back_tried_success[0]+=int(tds[0].get_text())
                back_tried_success[1]+=int(tds[1].get_text())
                quick_tried_success[0]+=int(tds[6].get_text())
                quick_tried_success[1]+=int(tds[7].get_text())
                quickopen_tried_success[0]+=int(tds[12].get_text())
                quickopen_tried_success[1]+=int(tds[13].get_text())

            data.append(back_tried_success[0]/sum(tried)*100)
            if back_tried_success[0] > 0:
                data.append(back_tried_success[1]/back_tried_success[0]*100)
            else:
                data.append("NA")
            data.append(quick_tried_success[0]/sum(tried)*100)
            if quick_tried_success[0] > 0:
                data.append(quick_tried_success[1]/quick_tried_success[0]*100)
            else:
                data.append('NA')
            data.append(quickopen_tried_success[0]/sum(tried)*100)
            if quickopen_tried_success[0]>0:
                data.append(quickopen_tried_success[1]/quickopen_tried_success[0]*100)
            else:
                data.append('NA')

            #4
            serve_tried_success = [0,0]
            dig_tried_success = [0,0]
            set_tried_success = [0,0]
            trs = tables[3].find('tbody').find_all('tr')
            for tr in trs:
                tds = tr.find_all('td')
                serve_tried_success[0]+=int(tds[0].get_text())
                serve_tried_success[1]+=int(tds[1].get_text())
                dig_tried_success[0]+=int(tds[5].get_text())
                dig_tried_success[1]+=int(tds[6].get_text())
                set_tried_success[0]+=int(tds[11].get_text())
                set_tried_success[1]+=int(tds[12].get_text())

            data.append(serve_tried_success[1]/serve_tried_success[0]*100)
            data.append(dig_tried_success[1]/dig_tried_success[0]*100)
            data.append(set_tried_success[1]/set_tried_success[0]*100)

            #5
            receive_tried_success = [0,0]
            receive_deviation = []
            blocking_tried_success_effective_assist = [0,0,0,0]
            penalty = 0
            error = 0
            trs = tables[4].find('tbody').find_all('tr')
            for tr in trs:
                tds = tr.find_all('td')
                receive_tried_success[0]+=int(tds[0].get_text())
                receive_deviation.append(int(tds[0].get_text()))
                receive_tried_success[1]+=int(tds[1].get_text())
                blocking_tried_success_effective_assist[0]+=int(tds[5].get_text())
                blocking_tried_success_effective_assist[1]+=int(tds[6].get_text())
                blocking_tried_success_effective_assist[2]+=int(tds[7].get_text())
                blocking_tried_success_effective_assist[3]+=int(tds[12].get_text())
                penalty+=int(tds[-2].get_text())
                error+=int(tds[-1].get_text())

            data.append(receive_tried_success[1]/receive_tried_success[0]*100)
            data.append(deviation(receive_deviation))
            data.append(blocking_tried_success_effective_assist[1]/blocking_tried_success_effective_assist[0]*100)
            data.append(blocking_tried_success_effective_assist[2]/blocking_tried_success_effective_assist[0]*100)
            data.append(blocking_tried_success_effective_assist[3]/blocking_tried_success_effective_assist[0]*100)
            data.append(penalty)
            data.append(error)
            
            if no ==0:
                if int(scores[0].get_text()) > int(scores[1].get_text()):
                    result = 1
                else:
                    result = 0
            else:
                if int(scores[0].get_text()) < int(scores[1].get_text()):
                    result = 1
                else:
                    result = 0
            data.append(result)

            attack_type = [open_tried_success[0], delay_tried_success[0], move_tried_success[0], back_tried_success[0], quick_tried_success[0], quickopen_tried_success[0]]
            data.insert(1,deviation(attack_type))

            counter+=1
            print(counter)
            writer.writerow(data)

In [ ]:
for i in range(100):
    writer.writerow(title)